In [2]:
import os
import time
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import FinNews as fn
import re

# Dash modules
import dash
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go

# Set up jupyter proxy
JupyterDash.infer_jupyter_proxy_config() 

%matplotlib inline

In [3]:
# Getting all file paths

path = r'assets/historical-symbols' # use your path
all_files = glob.glob(path + "/*.csv")

# Creating list to append all ticker dfs to
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concat all ticker dfs 
stock_df = pd.concat(li, axis=0, ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'])

available_indicators1 = list(stock_df['ticker'].unique())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [5]:
title_style = {'display': 'inline-block',
               'textAlign':'left',
               'verticalAlign':'center',
               'lineHeight':2.5,
               'height':75,
               'border': 'thick black solid',\
               'width':'85%',
               'backgroundColor': 'rgb(212, 150, 18)'}

title_link_style = {'display': 'inline-block',
               'textAlign':'center',
               'vertical-align':'center',
               'lineHeight':5,
               'height':75,
               'width':'10%',
               'float':'right',
               'border': 'thick black solid',\
               'fontSize':12,
               'backgroundColor': 'rgb(212, 150, 18)'}


tab_style = {'display': 'inline-block',
             'textAlign':'center',
             'verticalAlign':'top',
             'width':'15%',
             'border': 'thin lightgrey solid',
             'height':20}

blank_tab_style = {'display': 'inline-block',
                   'width':'15%',
                   'textAlign':'center',
                   'verticalAlign':'top',
                   'float':'right',
                   'border': 'thin lightgrey solid',
                   'fontSize':10,
                   'height':20}

ticker_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300}

chart_style = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':300}

portfolio_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':300}

chart_style_b = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':200}

portfolio_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'fontSize':12,
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':200}

news_style = {'fontSize':11,
              'color':'white',
              'textAlign':'left',
              'height':'auto',
#               'height':27,
              'margin':3,
              "lineHeight":1.5,
              'float':'middle'}

news_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300,'margin':0,'overflowY':'scroll',
                'color':'white',
                'backgroundColor':'black',
                'borderRadius': '.4rem'}

#                'vertical-align':'middle',

In [15]:
app = JupyterDash(__name__,suppress_callback_exceptions=True)

server = app.server

app.layout = html.Div([

    html.Div([
        html.H1('Dashboard Title',style=title_style),
        html.H3('Dashboard Info Link',style=title_link_style)
        ]),

    html.Div([dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='Portfolio Performance', value='tab-1'),
        dcc.Tab(label='Model Performance', value='tab-2'),
        dcc.Tab(label='Tab three', value='tab-3')
    ],style={'width':'50%','height':50})]),
    html.Div(id='tabs-example-content')
])

@app.callback(dash.dependencies.Output('tabs-example-content', 'children'),
              [dash.dependencies.Input('tabs-example', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            
                html.Div([
                    dcc.Dropdown(id='ticker',
                    options=[{'label': i, 'value': i} for i in available_indicators1],
                    value='CSCO') # the default is code_module AAA
                    ]),
                
                html.Div([
                    html.H2('Portfolio Performance',style=portfolio_style),
                    dcc.Graph(id='price_chart',style=chart_style),
                    html.Div(id='news_list',style=news_style_b)
                    ]),
                    
                html.Div([
                    html.H2('Other Portfolio Statistics',style=portfolio_style_b),
                    html.H2('News Info',style=chart_style_b),
                    ],style={'height':200})
    
                ])
                
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])
    
# Callback to connect input(s) to output(s)
@app.callback(dash.dependencies.Output('price_chart','figure'),
    [dash.dependencies.Input('ticker','value')])

# Step 3: Define the graph with plotly express
def update_ticker(ticker):
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=stock_df[stock_df['ticker']==ticker]['Date'],
                             y=stock_df[stock_df['ticker']==ticker]['Close'],
                            line={"color": "#228B22"},
                            mode="lines"))

    fig.update_layout(title_text=f'{ticker} Closing Price',title_x=0.5,
                         template="plotly_dark",font=dict(size=10),xaxis_showgrid=False,
                         yaxis_title="Closing Price",margin={"r": 20, "t": 35, "l": 20, "b": 10})

    return fig

# Creating callback to get news when ticker changes
@app.callback(dash.dependencies.Output('news_list', 'children'),
              [dash.dependencies.Input('ticker', 'value')])
def update_news(ticker):
    
    seeking_alpha = fn.SeekingAlpha(topics=['$'+ticker], save_feeds=True)
    
    news = seeking_alpha.get_news()
    
#     title_strings = [re.sub('[^A-Za-z0-9,\s]+', '', item['title']).lower().replace(" ", "-") for item in news[:5]]
#     url_ids = [re.sub('MarketCurrent:','news/',item) for item in news[:5]]
    
    news_info = html.Div(f'News for {ticker}',style={'backgroundColor':'gray'}),\
                    dbc.ListGroup([
                        dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[0]['title'],style=news_style),\
                                        href=(news[0]['link'])),\
                                        html.A(html.P(news[0]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[1]['title'],style=news_style),\
                                        href=(news[1]['link'])),\
                                        html.A(html.P(news[1]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[2]['title'],style=news_style),\
                                        href=(news[2]['link'])),\
                                        html.A(html.P(news[2]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[3]['title'],style=news_style),\
                                        href=(news[3]['link'])),\
                                        html.A(html.P(news[3]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[4]['title'],style=news_style),\
                                        href=(news[4]['link'])),\
                                        html.A(html.P(news[4]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    ],flush=True)
                    
    return news_info

if __name__ == '__main__':
    app.run_server(debug=True,port=8053)

Dash app running on http://127.0.0.1:8053/


In [6]:
seeking_alpha = fn.SeekingAlpha(topics=['$'+'CSCO'], save_feeds=True)
    
news = seeking_alpha.get_news()

news[1:3]

[{'title': 'Cisco Systems bags $1.18B defense IT contract',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://seekingalpha.com/api/sa/combined/CSCO.xml',
   'value': 'Cisco Systems bags $1.18B defense IT contract'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://seekingalpha.com/symbol/CSCO/news?source=feed_symbol_CSCO'}],
  'link': 'https://seekingalpha.com/symbol/CSCO/news?source=feed_symbol_CSCO',
  'id': 'https://seekingalpha.com/MarketCurrent:3706232',
  'guidislink': False,
  'published': 'Mon, 14 Jun 2021 17:34:05 -0400',
  'published_parsed': time.struct_time(tm_year=2021, tm_mon=6, tm_mday=14, tm_hour=21, tm_min=34, tm_sec=5, tm_wday=0, tm_yday=165, tm_isdst=0),
  'sa_author_name': 'Manshi Mamtora, CFA',
  'media_thumbnail': [{'url': ''}],
  'href': '',
  'sa_picture': '',
  'sa_symbol': 'CSCO',
  'sa_company_name': 'Cisco Systems, Inc.',
  'sa_stock': '',
  'topic': '$CSCO'},
 {'title': 'Cisco Systems, Inc. (CSCO

In [7]:
# [item['title'] for item in news[:5]]

[re.sub('[^A-Za-z0-9,\s]+', '', item['title']).lower().replace(" ", "-") for item in news[:5]]

['evercore-sees-ciscos-defense-deal-adding-to-tailwinds-of-growth',
 'cisco-systems-bags-118b-defense-it-contract',
 'cisco-systems,-inc-csco-management-presents-at-morgan-stanley-6th-annual-virtual-sustainable-futures-conference-transcript',
 'cisco-systems-rolls-out-reformulated-webex-suite',
 'not-in-a-hurry-to-sell-cisco']

In [63]:
news[1]['id']

'https://seekingalpha.com/MarketCurrent:3706232'

In [9]:
re.sub('MarketCurrent:','news/','https://seekingalpha.com/MarketCurrent:3706232')

'https://seekingalpha.com/news/3706232'

In [12]:
[item for item in news[:1]]

[{'title': "Evercore sees Cisco's defense deal 'adding to tailwinds' of growth",
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://seekingalpha.com/api/sa/combined/CSCO.xml',
   'value': "Evercore sees Cisco's defense deal 'adding to tailwinds' of growth"},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://seekingalpha.com/symbol/CSCO/news?source=feed_symbol_CSCO'}],
  'link': 'https://seekingalpha.com/symbol/CSCO/news?source=feed_symbol_CSCO',
  'id': 'https://seekingalpha.com/MarketCurrent:3706912',
  'guidislink': False,
  'published': 'Wed, 16 Jun 2021 09:57:19 -0400',
  'published_parsed': time.struct_time(tm_year=2021, tm_mon=6, tm_mday=16, tm_hour=13, tm_min=57, tm_sec=19, tm_wday=2, tm_yday=167, tm_isdst=0),
  'sa_author_name': 'Jason Aycock',
  'media_thumbnail': [{'url': ''}],
  'href': '',
  'sa_picture': '',
  'sa_symbol': 'CSCO',
  'sa_company_name': 'Cisco Systems, Inc.',
  'sa_stock': '',
  'topic': '$CSCO'}]

In [16]:
news[2]

{'title': 'Cisco Systems, Inc. (CSCO) Management Presents at Morgan Stanley 6th Annual Virtual Sustainable Futures Conference (Transcript)',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://seekingalpha.com/api/sa/combined/CSCO.xml',
  'value': 'Cisco Systems, Inc. (CSCO) Management Presents at Morgan Stanley 6th Annual Virtual Sustainable Futures Conference (Transcript)'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://seekingalpha.com/article/4433995-cisco-systems-inc-csco-management-presents-morgan-stanley-6th-annual-virtual-sustainable?source=feed_symbol_CSCO'}],
 'link': 'https://seekingalpha.com/article/4433995-cisco-systems-inc-csco-management-presents-morgan-stanley-6th-annual-virtual-sustainable?source=feed_symbol_CSCO',
 'id': 'https://seekingalpha.com/Article:4433995',
 'guidislink': False,
 'published': 'Wed, 09 Jun 2021 17:11:06 -0400',
 'published_parsed': time.struct_time(tm_year=2021, tm_mon=6, tm_mday=9, tm_hou

In [17]:
# Could create logic to swap regex dependent on if article or Market Current
[re.sub('MarketCurrent:|Article:','news/',item['id']) for item in news[:5]]

['https://seekingalpha.com/news/3706912',
 'https://seekingalpha.com/news/3706232',
 'https://seekingalpha.com/news/4433995',
 'https://seekingalpha.com/news/3704281',
 'https://seekingalpha.com/news/4433648']

In [73]:
import re
re.sub('[^A-Za-z0-9,\s]+', '', news[1]['title']).lower().replace(" ", "-")

'cisco-systems-bags-118b-defense-it-contract'

In [ ]:
re.match('[:+]')

re.search(r":\s(.*)", subject)

### Notes
1. How do I set a relative height
2. Could add ticker - https://community.plotly.com/t/strip-ticker-label/48348/6
3. Fix overlap or overflow of text in boxes for news


In [9]:
ticker = 'CSCO'

In [10]:
news_info = html.Div(f'News for {ticker}',style={'backgroundColor':'gray'}),\
                    dbc.ListGroup([
                        dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[0]['title'],style=news_style),\
                                        href=(news[0]['link'])),\
                                        html.A(html.P(news[0]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[1]['title'],style=news_style),\
                                        href=(news[1]['link'])),\
                                        html.A(html.P(news[1]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[2]['title'],style=news_style),\
                                        href=(news[2]['link'])),\
                                        html.A(html.P(news[2]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[3]['title'],style=news_style),\
                                        href=(news[3]['link'])),\
                                        html.A(html.P(news[3]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[4]['title'],style=news_style),\
                                        href=(news[4]['link'])),\
                                        html.A(html.P(news[4]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    ],flush=True)

In [10]:
from dashboard_pages import portfolio_performance

Hello
